# Stock Market Data Analysis with Alpha Vantage

This notebook provides a simplified way to fetch, transform, and visualize stock data using the Alpha Vantage API.

## Setup and Imports

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime

# Set plotting style
sns.set_theme(style="darkgrid")
plt.rcParams["figure.figsize"] = (12, 8)

# Configuration
API_KEY = "YOUR_API_KEY_HERE"  # Replace with your Alpha Vantage API key
SYMBOLS = ["SAP.DEX", "ALV.DEX", "BMW.DEX"]  # German stocks examples

## Fetch Stock Data

In [ ]:
def fetch_stock_data(symbol, time_period="daily", outputsize="full"):
    """Fetch stock data from Alpha Vantage API"""
    
    # Map time period to the appropriate API function
    function_map = {
        "daily": "TIME_SERIES_DAILY",
        "weekly": "TIME_SERIES_WEEKLY",
        "monthly": "TIME_SERIES_MONTHLY"
    }
    function = function_map.get(time_period.lower(), "TIME_SERIES_DAILY")
    
    # Build parameters
    params = {
        "function": function,
        "symbol": symbol,
        "apikey": API_KEY,
        "outputsize": outputsize
    }
    
    print(f"Fetching {time_period} data for {symbol}...")
    
    try:
        # Make API request
        response = requests.get("https://www.alphavantage.co/query", params=params)
        response.raise_for_status()
        data = response.json()
        
        # Check for error messages
        if "Error Message" in data:
            print(f"API Error: {data['Error Message']}")
            return None
        
        # Extract the time series data key
        time_series_key = f"Time Series ({time_period.capitalize()})"
        if time_period.lower() == "daily":
            time_series_key = "Time Series (Daily)"
            
        # Extract time series data
        time_series = data.get(time_series_key)
        if not time_series:
            print(f"No time series data found. Response: {data}")
            return None
        
        # Convert to DataFrame
        df = pd.DataFrame.from_dict(time_series, orient="index")
        
        # Clean up column names
        df.columns = [col.split('. ')[1] if '. ' in col else col for col in df.columns]
        df.columns = [col.lower() for col in df.columns]
        
        # Convert columns to numeric
        for col in df.columns:
            df[col] = pd.to_numeric(df[col])
        
        # Convert index to datetime and sort
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()
        
        print(f"Successfully fetched data for {symbol}")
        return df
        
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

In [ ]:
# Fetch data for our symbols
stock_data = {}
for symbol in SYMBOLS:
    df = fetch_stock_data(symbol)
    if df is not None:
        stock_data[symbol] = df
        print(f"\nFirst 5 rows of data for {symbol}:")
        display(df.head())
    time.sleep(1)  # Avoid API rate limits

## Data Transformation

In [ ]:
def calculate_metrics(df, ma_window=20):
    """Calculate various metrics for stock data"""
    
    # Create a copy to avoid modifying the original DataFrame
    df_metrics = df.copy()
    
    # Calculate daily returns
    df_metrics['daily_return'] = df_metrics['close'].pct_change() * 100
    
    # Calculate moving averages
    df_metrics[f'ma_{ma_window}'] = df_metrics['close'].rolling(window=ma_window).mean()
    
    # Calculate MACD components
    df_metrics['ema_12'] = df_metrics['close'].ewm(span=12, adjust=False).mean()
    df_metrics['ema_26'] = df_metrics['close'].ewm(span=26, adjust=False).mean()
    df_metrics['macd'] = df_metrics['ema_12'] - df_metrics['ema_26']
    df_metrics['signal'] = df_metrics['macd'].ewm(span=9, adjust=False).mean()
    
    # Calculate volatility
    df_metrics['volatility'] = df_metrics['daily_return'].rolling(window=ma_window).std()
    
    return df_metrics

# Calculate metrics for each stock
for symbol, df in stock_data.items():
    stock_data[symbol] = calculate_metrics(df)
    print(f"\nCalculated metrics for {symbol}:")
    display(stock_data[symbol].tail().round(2))  # Show the most recent data with metrics

## Visualization

In [ ]:
def plot_price_history(df, symbol, ma_window=20, start_date=None):
    """Plot stock price with moving average"""
    
    # Filter by start date if provided
    if start_date:
        df = df[df.index >= start_date]
    
    # Create the plot
    fig, ax = plt.subplots()
    ax.plot(df.index, df['close'], label='Close Price', linewidth=2)
    ax.plot(df.index, df[f'ma_{ma_window}'], label=f'{ma_window}-Day MA', linewidth=1.5, linestyle='--')
    
    # Customize the plot
    ax.set_title(f"{symbol} Price History")
    ax.set_xlabel('Date')
    ax.set_ylabel('Price')
    ax.legend()
    ax.grid(True)
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Plot price history for each stock
for symbol, df in stock_data.items():
    print(f"\nPlotting price history for {symbol}...")
    plot_price_history(df, symbol, start_date="2023-01-01")

In [ ]:
def plot_macd(df, symbol):
    """Plot MACD indicator"""
    
    # Create subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True, gridspec_kw={'height_ratios': [2, 1]})
    
    # Price chart on top subplot
    ax1.plot(df.index, df['close'], label='Close Price')
    ax1.set_title(f"{symbol} Price and MACD")
    ax1.set_ylabel('Price')
    ax1.legend()
    ax1.grid(True)
    
    # MACD chart on bottom subplot
    ax2.plot(df.index, df['macd'], label='MACD Line', color='blue')
    ax2.plot(df.index, df['signal'], label='Signal Line', color='red', linestyle='--')
    
    # MACD histogram
    hist_color = ['green' if val >= 0 else 'red' for val in (df['macd'] - df['signal'])]
    ax2.bar(df.index, df['macd'] - df['signal'], color=hist_color, alpha=0.5, label='Histogram')
    
    ax2.set_xlabel('Date')
    ax2.set_ylabel('MACD')
    ax2.legend()
    ax2.grid(True)
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Plot MACD for each stock
for symbol, df in stock_data.items():
    print(f"\nPlotting MACD for {symbol}...")
    plot_macd(df, symbol)

In [ ]:
def compare_stocks(stock_data, start_date=None):
    """Compare multiple stocks (normalized)"""
    
    # Create a DataFrame for comparison
    comparison_df = pd.DataFrame()
    
    for symbol, df in stock_data.items():
        # Filter by start date if provided
        if start_date:
            filtered_df = df[df.index >= start_date]
        else:
            filtered_df = df
        
        comparison_df[symbol] = filtered_df['close']
    
    # Normalize the data (starting from 100)
    normalized_df = comparison_df / comparison_df.iloc[0] * 100
    
    # Create the plot
    fig, ax = plt.subplots()
    
    for column in normalized_df.columns:
        ax.plot(normalized_df.index, normalized_df[column], label=column, linewidth=2)
    
    ax.set_title("Stock Price Comparison (Normalized to 100)")
    ax.set_xlabel('Date')
    ax.set_ylabel('Normalized Price')
    ax.legend()
    ax.grid(True)
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Compare our stocks
print("\nComparing stock performance...")
compare_stocks(stock_data, start_date="2023-01-01")

## Investment Insights

Based on the analysis above, we can draw several insights:

1. **Price Trends**: The charts show how each stock has performed over time, with moving averages helping to identify the overall trend.

2. **Volatility**: The volatility measurement helps identify which stocks have been more stable versus more volatile.

3. **MACD Signals**: The MACD indicator provides potential buy/sell signals when the MACD line crosses above/below the signal line.

4. **Comparative Performance**: The normalized comparison chart helps identify which stocks have outperformed others during the analyzed period.

Further analysis could include:
- Correlation between these stocks
- Risk-adjusted return metrics
- Comparison with broader market indices
- Fundamental analysis to complement the technical view